In [1]:
import numpy as np
import tensorflow as tf
import req
import time


In [2]:
class Client():
    def __init__(self, name, X, y, learning_rate):
        self.name = name
        self.X = tf.constant(X)
        self.y = tf.constant(y)
        self.W = tf.Variable(np.zeros((2, 1)))
        self.b = tf.Variable(0, dtype=tf.double)
        self.round = 0
        self.epochs = 100
        self.learning_rate = learning_rate
    
    def check_trian(self):
        curr_r = req.get_round()
        if curr_r > self.round:
            self.round = curr_r
            self.update_local_model()
            self.train()
            self.send_local_model()
            print("Trained global model.")
        else:
            print("Round is stil going on.")
    
    def update_local_model(self):
        global_p = req.get_pre_global()
        W, b = global_p['W'], global_p['b']
        self.W = tf.Variable(np.array(W).reshape((2, 1)))
        self.b = tf.Variable(b, dtype=tf.double)

    def train(self):
        for _ in range(self.epochs):
            with tf.GradientTape() as tape:
                tape.watch([self.W, self.b])
                phi = self.X @ self.W + self.b
                pred = tf.keras.activations.sigmoid(phi)
                loss = tf.keras.losses.binary_crossentropy(self.y, pred, axis=0)

            grad = tape.gradient(loss, {
                "W" : self.W,
                "b" : self.b
            })
            self.W = self.W - self.learning_rate * grad['W']
            self.b = self.b - self.learning_rate * grad['b']
        print(loss.numpy()[0])
    
    def convert_params(self):
        W_temp = self.W.numpy().tolist()
        W = [w[0] for w in W_temp]
        b = self.b.numpy().tolist()
        return W, b
    
    def send_local_model(self):
        param_id = "local_" + str(self.round) + "_" + self.name
        W, b = self.convert_params()
        req.post_param(param_id, W, b)
            
        

In [3]:
data = np.load('./data2d.npz')
X, y = data['X'], data['y']

In [4]:
client_1 = Client("1", X[:len(X)//2], y[:len(X)//2].reshape((35, 1)), 1e-2)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-11-23 15:51:39.856127: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-23 15:51:39.856621: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
client_1.check_trian()

0.22356791057917025
Trained global model.


In [6]:
for _ in range(100):
    time.sleep(3)
    client_1.check_trian()
    

0.15092891155227667
Trained global model.
Round is stil going on.
0.11562703108504209
Trained global model.
Round is stil going on.
0.09555254859231463
Trained global model.
0.08236076460241712
Trained global model.
Round is stil going on.
0.0729509349016698
Trained global model.
Round is stil going on.
0.06585608583166547
Trained global model.
0.060288499929816365
Trained global model.
Round is stil going on.
0.05578551888109767
Trained global model.
Round is stil going on.
0.05205667119889489
Trained global model.
0.04890990117056358
Trained global model.
Round is stil going on.
0.046212854412413136
Trained global model.
Round is stil going on.
0.043871173122425125
Trained global model.
0.04181565798587883
Trained global model.
Round is stil going on.
0.03999432992610858
Trained global model.
Round is stil going on.
0.03836733442276629
Trained global model.
0.03690356415107003
Trained global model.
Round is stil going on.
0.035578357662739625
Trained global model.
Round is stil going

KeyboardInterrupt: 

In [ ]:
req.get_round()

1